# 데이터 로드

In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/data/dacon/'

In [41]:
train_df = pd.read_csv(os.path.join(path + 'train.csv'), encoding='utf=8')
test_df =  pd.read_csv(os.path.join(path + 'test.csv'), encoding='utf=8')
submission = pd.read_csv(os.path.join(path + 'sample_submission.csv'), encoding='utf=8')

In [6]:
print(train_df.shape)
train_df.head()

(1205, 12)


,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   object 
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   object 
 8   중식메뉴            1205 non-null   object 
 9   석식메뉴            1205 non-null   object 
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 113.1+ KB


# 1차 학습<br/>
수치 데이터로만 기본 학습

## 요일 feature 숫자 매핑

In [45]:
train_df['요일'] = train_df['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
test_df['요일'] = test_df['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})

In [12]:
train_df.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계'],
      dtype='object')

## MAE 평가지표 함수 생성

In [38]:
from sklearn.metrics import mean_absolute_error

def get_mae(model):
  pred1 = model1.predict(X_test) # 중식계
  pred2 = model2.predict(X_test) # 석식계

  mae1 = mean_absolute_error(y1_test, pred1)
  mae2 = mean_absolute_error(y2_test, pred2)

  print(f"중식계 MAE: {mae1:.3f}")
  print(f"석식계 MAE: {mae2:.3f}")
  return mae1, mae2

## RandomForestRegressor 모델 학습/예측/평가

In [20]:
from sklearn.model_selection import train_test_split

y1_target = train_df['중식계']
y2_targwt = train_df['석식계']

X_features = train_df[['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수','현본사소속재택근무자수']]

X_train, X_test, y1_train, y1_test = train_test_split(X_features, y1_target, test_size = 0.2, random_state=216)
X_train, X_test, y2_train, y2_test = train_test_split(X_features, y1_target, test_size = 0.2, random_state=216)

In [39]:
from sklearn.ensemble import RandomForestRegressor

model1 = RandomForestRegressor(n_jobs=-1, random_state=216)
model2 = RandomForestRegressor(n_jobs=-1, random_state=216)

# 학습
model1.fit(X_train, y1_train) 
model2.fit(X_train, y2_train)

# 예측 및 평가
models = [model1, model2]
get_mae(models)

중식계 MAE: 71.172
석식계 MAE: 71.172


(71.17157676348548, 71.17157676348548)

## submission csv 생성

In [46]:
X_train = train_df[['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수','현본사소속재택근무자수']]

y1_train = train_df['중식계']
y2_train = train_df['석식계']

X_test = test_df[['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수','현본사소속재택근무자수']]

In [47]:
# 학습
model1.fit(X_train, y1_train) 
model2.fit(X_train, y2_train)

# 예측
pred1 = model1.predict(X_test) # 중식계
pred2 = model2.predict(X_test) # 석식계

In [49]:
submission['중식계'] = pred1
submission['석식계'] = pred2

submission.to_csv('Submission_1.csv', index=False)

# 2차 학습

중식메뉴/ 석식메뉴 feature화